In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys,os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

######Load in functions
sys.path.append("../python/")
import get_counts_inline as gc
# from LL import ll

%load_ext Cython

In [50]:
%%cython

###############################################################################
# LL.pyx
###############################################################################
#
# Evaluate the likelihood as defined in 1705.00009, accounting for the minus
# error. For the moment the prior term has been left out
#
# Mask whatever you want
#
# Written: Nick Rodd, MIT, 5 August 2017
# Modified: Ben Safdi,UM, 15 August 2017
#
###############################################################################


# Import basic functions
import numpy as np
cimport numpy as np
cimport cython


import get_counts_inline as gc
cimport get_counts_inline as gc

DTYPE = np.float
ctypedef np.float_t DTYPE_t

# C functions
cdef extern from "math.h":
    double pow(double x, double y) nogil
    double sqrt(double x) nogil
    double cos(double x) nogil
    double tgamma(double x) nogil
    double exp(double x) nogil
    double sin(double x) nogil
    double fabs(double x) nogil
    double tan(double x) nogil

    double log(double x) nogil
    double lgamma(double x) nogil

cdef double pi = np.pi



@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
@cython.initializedcheck(False)
cpdef double ll(double[:, :, ::1] PSR_data, double[:, :, ::1] omega_ijk, 
                double Nbulge, double Ndisk, double alpha, double n, 
                double sigma, double z0, double beta_disk, double beta_bulge, double rcut = 3.0, double Lmin = 1.0e31, double Lmax_disk = 1.0e36,double Lmax_bulge = 1.0e36,int Ns = 200, int Nang = 5,double smax_disk = 20) nogil:
    """ Calculate the likelihood as a function of the bulge and disk params
    """

    # Setup loop variables
    cdef double omega_val, Nmodel, Nobs, N_bulge, N_disk
    cdef Py_ssize_t i, j, k
    
    # Now calculate the likelihood over all bins
    cdef double ll = 0.
    for i in range(6,7): # loop over longitude
        for j in range(6,7): # loop over latitude
            for k in range(1): # loop over flux
                omega_val = omega_ijk[i,j,k] 

                N_disk = gc.Ndisk_full_ang_ijk(i,j,k,Ndisk, omega_val,n,sigma,z0,beta_disk,Lmin,Lmax_disk,Ns,Nang,smax_disk)
                N_bulge = gc.Nbulge_full_ang_ijk(i,j,k,Nbulge, omega_val,alpha,beta_bulge,rcut, Lmin,Lmax_bulge,Ns,Nang)

                Nmodel = N_disk #+ N_bulge

                Nobs = PSR_data[i,j,k]


                ll += Nobs*log(Nmodel) - Nmodel #- lgamma(Nobs + 1)
                
    return ll

In [51]:
# Fix base parameters
Nb=0.
n=2.35
sigma=1.528
alpha=2.6
z0=0.7
beta=1.2

In [52]:
# Load the binned efficiency and data files
PSR_data = np.load('../data/PSR_data.npy') + np.load('../data/PSR_data_3fgl.npy')
omega_ijk = np.load('../data/omega_ijk.npy')


In [53]:
# %%timeit
print ll(PSR_data, omega_ijk, 1, 1, alpha, n, sigma, z0, beta, beta)

-3.69281951166e-06
